<a href="https://colab.research.google.com/github/kyungmin0101/medical_ai/blob/main/240220/Medical_Ai_day_5_Hyunsang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [131]:
import glob
import csv
import pandas as pd

#All of 1stSeries data
path1 = "/content/drive/MyDrive/Refractive Index Measurements New/1stSeries_07.05.21/*/{}*".format('W') #training set
df_list_1st = []
ri_list_1st = []
for file in glob.glob(path1, recursive = True) :
  f = open(file,'rt')
  reader = csv.reader(f)
  csv_context = []
  for raw in reader :
    csv_context.append(raw)
  f.close()

  df_list_1st.append(pd.DataFrame(csv_context)[24:])
  ri_list_1st.append(float(file.split('/')[-2]))

#All of 2ndSeries data
path2 = "/content/drive/MyDrive/Refractive Index Measurements New/2ndSeries_21.05.21/*/{}*".format('W')#test set
df_list_2nd = []
ri_list_2nd = []
for file in glob.glob(path2, recursive = True) :
  f = open(file,'rt')
  reader = csv.reader(f)
  csv_context = []
  for raw in reader :
    csv_context.append(raw)
  f.close()

  df_list_2nd.append(pd.DataFrame(csv_context)[24:])
  ri_list_2nd.append(float(file.split('/')[-2]))

In [129]:
len(ri_list_1st)

231

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import numpy as np

x_train_2nd, x_test_2nd, y_train_2nd, y_test_2nd = train_test_split(df_list_2nd, ri_list_2nd, test_size=0.1)

nsamples, nx, ny = np.array(df_list_1st).shape
x_train = np.array(df_list_1st).reshape((nsamples,nx*ny))

nsamples, nx, ny = np.array(x_test_2nd).shape
x_test = np.array(x_test_2nd).reshape((nsamples,nx*ny))

rf_regressor = RandomForestRegressor(n_estimators=100, random_state=40)
svr_model = SVR(kernel='rbf', C=0.01, gamma=0.1)

rf_regressor.fit(x_train, ri_list_1st)
svr_model.fit(x_train, ri_list_1st)

xgb_regressor = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1,
max_depth=2, alpha=10, n_estimators=100, random_state=0)
xgb_regressor.fit(x_train, ri_list_1st)

lgb_regressor = lgb.LGBMRegressor(objective='regression', num_leaves=31, learning_rate=0.05, n_estimators=100)
lgb_regressor.fit(x_train, ri_list_1st)


In [133]:
rf_pred = rf_regressor.predict(x_test)
svr_pred = svr_model.predict(x_test)
xgb_pred = xgb_regressor.predict(x_test)
lgb_pred = lgb_regressor.predict(x_test)

#rf
mse = mean_squared_error(y_test_2nd, rf_pred)
print(f'RF Mean Squared Error: {mse}')
r2_value = r2_score(y_test_2nd, rf_pred)
print(f'RF R-squared: {r2_value}')

#svr
mse = mean_squared_error(y_test_2nd, svr_pred)
print(f'SVR Mean Squared Error: {mse}')
r2_value = r2_score(y_test_2nd, svr_pred)
print(f'SVR R-squared: {r2_value}')

#XGB
mse = mean_squared_error(y_test_2nd,xgb_pred)
print(f'XGB Mean Squared Error: {mse}')
r2_value = r2_score(y_test_2nd, xgb_pred)
print(f'XGB R-squared: {r2_value}')

#LGBM
mse = mean_squared_error(y_test_2nd,lgb_pred)
print(f'LGBM Mean Squared Error: {mse}')
r2_value = r2_score(y_test_2nd, lgb_pred)
print(f'LGBM R-squared: {r2_value}')

RF Mean Squared Error: 0.007692282916666582
RF R-squared: -1.1393244942300988
SVR Mean Squared Error: 0.003604166666666662
SVR R-squared: -0.0023658925208829285
XGB Mean Squared Error: 0.003604166805744734
XGB R-squared: -0.002365931200313698
LGBM Mean Squared Error: 0.006526662317566057
LGBM R-squared: -0.8151501592960524
